In [2]:
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars C://Users//Bubrick//Downloads//pyth//sqlite-jdbc-3.23.1.jar'
import findspark
findspark.init()
import pyspark
import sqlite3
from pyspark.sql import functions as fun
from pyspark.sql import SQLContext
from pyspark import SparkContext

sc = SparkContext()
sqlContext = SQLContext(sc)

In [64]:
df = sqlContext.read.jdbc("jdbc:sqlite:task2.db", 'DATA') 

#spark = SparkSession.builder.appName("PySpark").getOrCreate()
# props
# props.put("driver", "org.sqlite.JDBC")
# sqlContext.read.jdbc("jdbc:sqlite:task2.db", props)

# df = sqlContext.read.format('jdbc').options(url='jdbc:sqlite:task2.db',
#                                             dbtable='DATA',driver='org.sqlite.JDBC').load()

In [65]:
df.printSchema()

root
 |-- REGION: string (nullable = true)
 |-- TIK: string (nullable = true)
 |-- UIK: string (nullable = true)
 |-- VOTERS: long (nullable = true)
 |-- BULLETINS: long (nullable = true)
 |-- ANTICIPATORY: long (nullable = true)
 |-- INSIDE: long (nullable = true)
 |-- OUTSIDE: long (nullable = true)
 |-- CANCEL: long (nullable = true)
 |-- PORTABLE: long (nullable = true)
 |-- STATIONARY: long (nullable = true)
 |-- INVALID: long (nullable = true)
 |-- VALID: long (nullable = true)
 |-- LOST: long (nullable = true)
 |-- UNACCOUNTED: long (nullable = true)
 |-- BABURIN: long (nullable = true)
 |-- GRUDININ: long (nullable = true)
 |-- JIRINOVSKIY: long (nullable = true)
 |-- PUTIN: long (nullable = true)
 |-- SOBCHAK: long (nullable = true)
 |-- SURAYKIN: long (nullable = true)
 |-- TITON: long (nullable = true)
 |-- YAVLINSKIY: long (nullable = true)



найти явку (%) по всем регионам, результат отсортировать по убыванию

In [66]:
df = df.withColumn('TOTAL', df['INVALID'] + df['VALID'])

tempdf = df \
    .groupBy('REGION') \
    .agg(fun.sum('VOTERS'), fun.sum('TOTAL'))
    
tempdf.withColumn('%', tempdf['sum(TOTAL)'] / tempdf['sum(VOTERS)'] * 100) \
    .select('REGION', '%') \
    .orderBy('%', ascending=False) \
    .show(100)

+--------------------+------------------+
|              REGION|                 %|
+--------------------+------------------+
|Территория за пре...| 98.01821236184206|
|     Республика Тыва| 93.62542974951742|
|Ямало-Ненецкий ав...| 91.87213306617981|
|Кабардино-Балкарс...|  91.6978754085207|
|Чеченская Республика| 91.50929703298173|
|Республика Северн...| 89.93704831730909|
| Республика Дагестан| 87.44351419820495|
|Карачаево-Черкесс...|   87.343809390099|
| Кемеровская область| 83.07556958224816|
|Чукотский автоном...| 82.26051697921946|
|Республика Ингушетия| 81.95937165135314|
|    Брянская область| 79.65976807571519|
|   Тюменская область|  78.8829673763749|
| Республика Мордовия| 77.83648793754097|
|  Краснодарский край| 77.83466166202626|
|Республика Татарс...|  77.3652654820273|
|Чувашская Республ...| 76.21061419670117|
|Республика Башкор...| 75.42120065745192|
|  Республика Бурятия| 75.19133382510174|
|Республика Адыгея...|  74.3059515324305|
| Ставропольский край| 73.77007114

выбрать любимого кандидата и найти тот избиратльный участок, на котором он получил наибольший результат (учитывать участки на которых проголосовало больше 300 человек)

In [45]:
df.filter(df['TOTAL'] > 300) \
    .withColumn('JIRINOVSKIY BEST RES', df['JIRINOVSKIY'] / df['TOTAL']) \
    .select('REGION', 'TIK', 'UIK', 'JIRINOVSKIY BEST RES', 'TOTAL') \
    .orderBy('JIRINOVSKIY BEST RES', ascending=False) \
    .show(1)

+--------------------+--------------------+---------+--------------------+-----+
|              REGION|                 TIK|      UIK|JIRINOVSKIY BEST RES|TOTAL|
+--------------------+--------------------+---------+--------------------+-----+
|Новгородская область|Новгородcкая горо...|УИК №1194|  0.3791469194312796|  422|
+--------------------+--------------------+---------+--------------------+-----+
only showing top 1 row



найти регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна

In [50]:
tempdf = df.groupBy('TIK').sum('VOTERS','TOTAL')

tempdf = tempdf.withColumn('%', tempdf['sum(TOTAL)'] / tempdf['sum(VOTERS)'] * 100)
tempdf = df.select('REGION', 'TIK') \
    .distinct() \
    .join(tempdf, 'TIK')
tempdf = tempdf.groupBy('REGION') \
    .agg(fun.max('%').alias('max_%'), fun.min('%').alias('min_%'))
tempdf.withColumn('DIFFERENCE', tempdf['max_%'] - tempdf['min_%']) \
    .orderBy('DIFFERENCE', ascending=False) \
    .select('REGION', 'DIFFERENCE') \
    .show(1)

+--------------------+----------------+
|              REGION|      DIFFERENCE|
+--------------------+----------------+
|Архангельская обл...|49.8591648406849|
+--------------------+----------------+
only showing top 1 row



посчитать дисперсию по явке для каждого региона (учитывать УИК)

In [54]:
df.withColumn('%', 100 * df['TOTAL'] / df['VOTERS']) \
    .groupBy('REGION') \
    .agg(fun.stddev('%').alias('STDDEV')) \
    .orderBy('STDDEV', ascending=False) \
    .show(100)

+--------------------+------------------+
|              REGION|            STDDEV|
+--------------------+------------------+
| Сахалинская область|20.188798623588145|
|     Камчатский край| 16.87347577751214|
|  Мурманская область| 16.74213145389531|
|     Приморский край|16.639015831512317|
|Республика Адыгея...| 16.33324959884359|
|   Самарская область|15.628836277115965|
| Саратовская область|15.426531267153354|
| Республика Калмыкия|15.350844523090664|
| Воронежская область|14.975754071870506|
| Магаданская область|14.387651017723126|
|Республика Татарс...|14.172353736111935|
|    Липецкая область|14.111877115838016|
|Архангельская обл...|14.094552956128387|
|  Краснодарский край|13.946578225528093|
|    Хабаровский край|13.811595897262203|
|Белгородская область|13.758282414787915|
| Ставропольский край|13.411190491014079|
|  Республика Бурятия|13.308253052737115|
|  Ростовская область|13.138651268427406|
|Чувашская Республ...| 13.04162760122739|
| Республика Дагестан|12.954393980

для каждого кандидата посчитать таблицу: результат (%, округленный до целого) - количество УИК, на которых кандидат получил данный результат

In [72]:
cands =  df.columns[15:23]

In [74]:
for cand in cands:
    print(cand + ':')
    df.withColumn('%', fun.round(100 * df[cand] / df['TOTAL'])) \
        .na.fill(0.0) \
        .groupBy('%') \
        .count() \
        .orderBy('%') \
        .show(100)

BABURIN:
+----+-----+
|   %|count|
+----+-----+
| 0.0|45115|
| 1.0|47940|
| 2.0| 3704|
| 3.0|  556|
| 4.0|  179|
| 5.0|   89|
| 6.0|   35|
| 7.0|   20|
| 8.0|   22|
| 9.0|   10|
|10.0|    7|
|11.0|    5|
|12.0|    1|
|13.0|    1|
|14.0|    3|
|15.0|    1|
|17.0|    1|
|18.0|    2|
|20.0|    3|
|21.0|    1|
|23.0|    1|
|44.0|    1|
+----+-----+

GRUDININ:
+----+-----+
|   %|count|
+----+-----+
| 0.0| 1181|
| 1.0| 2164|
| 2.0| 3073|
| 3.0| 3425|
| 4.0| 3655|
| 5.0| 3863|
| 6.0| 4001|
| 7.0| 4543|
| 8.0| 5278|
| 9.0| 6087|
|10.0| 7060|
|11.0| 7735|
|12.0| 7891|
|13.0| 7440|
|14.0| 6116|
|15.0| 4861|
|16.0| 3718|
|17.0| 2925|
|18.0| 2352|
|19.0| 1887|
|20.0| 1519|
|21.0| 1283|
|22.0| 1057|
|23.0|  885|
|24.0|  765|
|25.0|  573|
|26.0|  440|
|27.0|  358|
|28.0|  298|
|29.0|  217|
|30.0|  170|
|31.0|  129|
|32.0|  106|
|33.0|  112|
|34.0|   70|
|35.0|   68|
|36.0|   58|
|37.0|   43|
|38.0|   51|
|39.0|   26|
|40.0|   31|
|41.0|   21|
|42.0|   22|
|43.0|   17|
|44.0|   15|
|45.0|   15|
|46.0